<a href="https://colab.research.google.com/github/PhoenixCC0722/Journey_to_become_DataScientist/blob/main/3_1_5_quality_assessment_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quality Assessment
Now it's your turn. Follow [the steps on the platform](https://learn.wbscodingschool.com/courses/data-science/lessons/%f0%9f%93%9d-5-data-quality/) and use what you've learnt to see how reliable the data is.

In [1]:
import pandas as pd

Load our cleaned DataFrames

In [4]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

## 1.&nbsp; Define Pandas display format

In [6]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # format floating-point numbers to two decimal places
pd.set_option('display.max_rows',1000)

## 2.&nbsp; Exclude unwanted orders

In [ ]:
# 1st, exclude the orders that were left in teh shopping cart or cancelled.
# 2nd, make sure that orders in the dataset are in both orders and orderlines
# orders –> Keep only orders with the states that you want to work with.
or_df = orders_cl.copy()
#print(or_df.state.unique()) # ['Cancelled' 'Completed' 'Pending' 'Shopping Basket' 'Place Order']
filt_or_df = or_df.loc[or_df.state.isin(['Completed','Pending','Place Order']),:]
#print(filt_or_df.state.unique())
# orderlines –> Keep only orders present in orders.
# orders –> Keep only orders present in orderlines.
ol_df = orderlines_cl.copy()
print(ol_df.columns, filt_or_df.columns, sep='\n')
ol_df = ol_df.rename(columns={'id_order':'order_id'})
merge_df = pd.merge(ol_df, filt_or_df, on='order_id', how='inner')
merge_df.head(20)

## 3.&nbsp; Exclude orders with unknown products


In [29]:
# wenn die sku not in the products Dataframe, then we delete the whole row in both the orders and orderline tables
pd_df = products_cl.copy()
merge_df_ec = merge_df.loc[merge_df.sku.isin(pd_df['sku']),:]
merge_df_ec.head()

,id,order_id,product_id,product_quantity,sku,unit_price,date,created_date,total_paid,state
0,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16,2017-01-01 01:51:47,51.48,Completed
1,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34,2017-01-01 01:57:34,18.99,Completed
2,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,2017-01-01 02:02:38,72.19,Completed
3,1119133,299555,0,1,BEA0065,256.49,2017-01-01 02:18:45,2017-01-01 02:18:45,263.48,Place Order
4,1119134,299556,0,1,CRU0039-A,60.90,2017-01-01 02:20:14,2017-01-01 02:30:08,65.89,Completed


# 4.  Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [31]:
# your code here
ol_df['unit_price_total'] = ol_df.unit_price * ol_df.product_quantity
ol_df.head()

,id,order_id,product_id,product_quantity,sku,unit_price,date,unit_price_total
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74


#### Step 2:
Group by `id_order`, summarising by the sum of `unit_price_total`

In [63]:
# id_order already renamed as order_id
gp_ol_df = ol_df.groupby('order_id')['unit_price_total'].sum()
gp_ol_df.head()

order_id
241319    44.99
241355   135.98
241423   129.16
242832    10.77
243330    77.99
Name: unit_price_total, dtype: float64

### What is the average difference between `total_paid` and `unit_price_total`?

In [64]:
# .sum() is only a series, have to change to dataframe
gp_ol_df = gp_ol_df.to_frame(name='unit_price_total')
mg_or_gp_ol = pd.merge(filt_or_df, gp_ol_df, how='left', on='order_id')
#mg_or_gp_ol.head()
mg_or_gp_ol['difference'] = mg_or_gp_ol.total_paid - mg_or_gp_ol.unit_price_total
#print(mg_or_gp_ol['difference'].mean()) # 3.0010141765989085
mg_or_gp_ol.head(25)

,order_id,created_date,total_paid,state,unit_price_total,difference
0,241423,2017-11-06 13:10:02,136.15,Completed,129.16,6.99
1,242832,2017-12-31 17:40:03,15.76,Completed,10.77,4.99
2,243330,2017-02-16 10:59:38,84.98,Completed,77.99,6.99
3,245275,2017-06-28 11:35:37,149.00,Completed,149.00,0.00
4,245595,2017-01-21 12:52:47,112.97,Completed,105.98,6.99
5,245851,2017-04-04 20:58:21,79.99,Pending,79.99,0.00
6,245941,2017-01-01 10:32:23,183.52,Completed,NaN,NaN
7,246018,2017-02-13 23:02:57,211.95,Completed,206.96,4.99
8,246405,2017-11-24 10:01:27,407.96,Completed,275.75,132.21
9,247524,2018-01-09 14:21:34,167.98,Completed,162.99,4.99


### What is the distribution of these differences?

In [65]:
# distribution
print(mg_or_gp_ol['difference'].value_counts().head(10), mg_or_gp_ol['difference'].value_counts(normalize=True).head(10), sep='\n')
#mg_or_gp_ol['difference'].nunique()

0.00    34143
6.99     6584
4.99     4699
4.99     3373
4.99     3254
3.99     2679
6.99     2619
3.99     2281
3.99     2102
6.99     1758
Name: difference, dtype: int64
0.00   0.43
6.99   0.08
4.99   0.06
4.99   0.04
4.99   0.04
3.99   0.03
6.99   0.03
3.99   0.03
3.99   0.03
6.99   0.02
Name: difference, dtype: float64


### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [66]:
# your explanation here
mg_or_gp_ol.loc[(mg_or_gp_ol['difference'] >= 7.0),:] #2861 rows × 6 columns

,order_id,created_date,total_paid,state,unit_price_total,difference
8,246405,2017-11-24 10:01:27,407.96,Completed,275.75,132.21
13,251688,2017-11-24 23:57:07,505.76,Completed,485.77,19.99
14,251969,2017-11-21 21:04:12,610.07,Completed,590.08,19.99
17,253306,2017-10-04 14:14:45,497.68,Completed,477.68,20.00
22,254537,2017-05-23 19:58:30,102.97,Pending,32.99,69.98
...,...,...,...,...,...,...
101652,527020,2018-03-14 12:55:02,506.98,Pending,499.98,7.00
101712,527125,2018-03-14 12:00:55,73.70,Place Order,53.71,19.99
101738,527167,2018-03-14 12:18:54,361.56,Pending,354.56,7.00
101810,527294,2018-03-14 13:08:14,182.96,Pending,175.96,7.00


### If there are differences that you can’t explain: what should you do with these orders?

In [ ]:
# your code here

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [ ]:
# your thoughts & code here

Do not forget to download your quality controlled DataFrames

In [ ]:
from google.colab import files

#orders_quality_controlled.to_csv("orders_qu.csv", index=False)
#files.download("orders_qu.csv")

#orderlines_quality_controlled.to_csv("orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")